<img src="../assets/notebook.png" />
<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Generative Models For Spatial-Based Bottom <br><br> Hypoxia Forecasting In The Black Sea</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

Do not forget to activate the **environnement esa_diffusion**

In [ ]:
# ---------- Librairies ----------
import os
import sys
import dawgz
import wandb
import xarray
import random
import numpy             as np
import pandas            as pd
import seaborn           as sns
import matplotlib.pyplot as plt

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim

# ---------- Jupyter ----------
%matplotlib inline
plt.rcParams.update({'font.size': 13})

# Making sure modules are reloaded when modified
%reload_ext autoreload
%autoreload 2

# Moving to code folder
%cd poseidon/

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Playground</center>
    </b>
</p>
<hr style="color:#5A7D9F;">